In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import pdfplumber
import pandas as pd
import re
from pathlib import Path

# Folder with PDFs
pdf_folder = Path("data24")

# Regex for result lines (Platz wird ignoriert)
"""line_pattern = re.compile(    
    r"^(?:[1-9]\d?|50)\s+"                                          
    r"(?P<leistung>("
    r"\d{1,2},\d{2}|"          # SS,SS
    r"\d{1,2}:\d{2},\d{2}|"    # M:SS,SS
    r"\d{1,2}:\d{2}:\d{2}|"    # H:MM:SS
    r"\d{1,2}:\d{2}|"           # M:SS, z.B. 29:00
    r"\d{1,3}(?:\.\d{3})?"       # Punkte, z.B. Zehnkampf 8307
    r"))\s*"                     # Leistung
    r"(?:(?P<wind>[+-]?\d,\d)\s+)?"              # Wind optional
    r"(?P<name>[A-Za-zÄÖÜäöüß ,\-]+)\s+"           # Name
    r"(?P<geburtsjahr>\d{2,4})\s+"                 # Geburtsjahr
    r"(?P<verein>[A-Za-zÄÖÜäöüß0-9 .\-()/]+)\s+"   # Verein
    #Für 01-17
    r"(?P<datum>\d{2}\.\d{2}\.(?:\d{2}|\d{4})?)\s+"
    # Für 18-22
    # r"(?P<datum>\d{2}\.\d{2}\.(?:\d{2}|\d{4}))?\s+"
    r"(?P<ort>[A-Za-zÄÖÜäöüß .\-()]+)"             # Ort
)"""

line_pattern = re.compile(
    r"^(?:[1-9]\d?|50)\s+"                              # Platz 1–50, zwingend
    r"(?P<leistung>\d{1,2},\d{2})\s+"                  # Leistung
    r"(?:(?P<wind>[+-]?\d,\d)\s+)?"                    # optionaler Wind
    r"(?P<name>[A-Za-zÄÖÜäöüß'´`\- ]+?)\s+"            # Name
    r"(?P<geburtsjahr>\d{4})"                          # Geburtsjahr
    r"(?P<verein>[A-Za-zÄÖÜäöüß0-9 .\-()/]+?)\s+"      # Verein, direkt danach möglich
    r"(?P<datum>\d{2}\.\d{2}\.\d{4})\s+"               # Datum
    r"(?P<ort>[A-Za-zÄÖÜäöüß .\-()]+)$"                # Ort
)

# Laufdisziplinen priorisiert (lange zuerst)
lauf_pattern = (
    r"10\s?km Straßengehen|20\s?km Straßengehen|50\s?km Straßengehen|"
    r"(?:10|20|50)\s?km\s?Gehen|(?:10|20|50)\s?k\s?Gehen|"
    r"(?:5|10|20|50|100)\s?km|"
    r"(?:1\.?000|1\.?500|2\.?000|3\.?000|5\.?000|10\.?000)\s?m\s?(?:Bahngehen|Gehen|Hindernis|Hürden)?|"
    r"(?:60|80|100|110|200|300|400|800|1000|1500|2000|3000|5000|10000)\s?m\s?(?:Hindernis|Hürden)?|"
    r"^(60|80|100|110|200|300|400|800|1000|1500|2000|3000|5000|10000)\s*[\u00A0\u202F]?\s*m\b"
    r"Halbmarathon|Marathon"
)

lauf_regex = re.compile(lauf_pattern, re.IGNORECASE)


# Komplette Disziplin-Regex
discipline_pattern = re.compile(
    r"^("
    + lauf_pattern + "|" +
    r"Weitsprung|Hochsprung|Dreisprung|Stabhochsprung|"
    r"Kugelstoß|Speerwurf|Diskuswurf|Hammerwurf|"
    r"Zehnkampf|Siebenkampf|Fünfkampf|10-Kampf|7-Kampf|5-Kampf"
    r")",
    re.IGNORECASE
)

discipline_standardization = {
    # Läufe
    "60m": "60 m", "100m": "100 m", "200m": "200 m", "300m": "300 m",
    "400m": "400 m", "800m": "800 m", "1000m": "1000 m", "1500m": "1500 m",
    "3000m": "3000 m", "5000m": "5000 m", "10000m": "10 000 m",
    "5km": "5 km", "10km": "10 km", "20km": "20 km",
    "10 km Straßengehen": "10 km Gehen",
    "10 km Gehen": "10 km Gehen",
    "20 km Gehen": "20 km Gehen",
    "20 km Straßengehen": "20 km Gehen",
    "5000 m Bahngehen": "5000 m Gehen",

    # Hürden
    "100 m Hürden": "100 m Hürden", "110 m Hürden": "110 m Hürden", "400 m Hürden": "400 m Hürden",

    # Sprung / Wurf
    "Weitsprung": "Weitsprung", "Hochsprung": "Hochsprung", "Dreisprung": "Dreisprung",
    "Stabhochsprung": "Stabhochsprung", "Kugelstoß": "Kugelstoß", "Speerwurf": "Speerwurf",
    "Diskuswurf": "Diskuswurf", "Hammerwurf": "Hammerwurf",

    # Mehrkampf
    "Zehnkampf": "Zehnkampf", "Siebenkampf": "Siebenkampf"
}

# Funktion zum Parsen von Dateinamen
def parse_filename(filename):
    stem = Path(filename).stem

    # Jahr
    year_match = re.search(r"bestenliste(\d{4})", stem)
    year = year_match.group(1) if year_match else ""

    # Geschlecht
    if any(s in stem for s in ["maennliche", "maenner", "junioren"]):
        gender = "M"
    elif any(s in stem for s in ["weibliche", "frauen", "juniorinnen"]):
        gender = "W"
    else:
        gender = ""

    # Altersklasse
    if "maenner" in stem:
        age_class = "Männer"
    elif "frauen" in stem:
        age_class = "Frauen"
    elif "junioren" in stem or "juniorinnen" in stem:
        age_class = "U23"
    else:
        u_match = re.search(r"U\d{2}$", stem)
        if u_match:
            age_class = u_match.group(0)
        else:
            num_match = re.search(r"(\d{1,2})$", stem)
            age_class = num_match.group(1) if num_match else ""

    return year, gender, age_class

# Funktion zum Ersetzen von Umlauten
def replace_umlauts(text):
    if not isinstance(text, str):
        return text
    replacements = {
        "ä": "ae", "ö": "oe", "ü": "ue",
        "Ä": "Ae", "Ö": "Oe", "Ü": "Ue",
        "ß": "ss"
    }
    for orig, repl in replacements.items():
        text = text.replace(orig, repl)
    return text

# Parse PDF
def parse_pdf(pdf_path):
    year, gender, age_class = parse_filename(pdf_path.name)

    results = []
    current_discipline = None
    ignore_section = False
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if not text:
                continue
            for line in text.split("\n"):
                line = line.strip()
                if not line:
                    continue
                if line.startswith("Ausländer"):
                    ignore_section = True
                    continue
                if re.search(r"(?i)(mannschaft|staffel|team|\d+\s?x\s?\d+\s?m)", line):
                    continue
                
                # Wenn wir im Ausländer-Abschnitt sind, bis zur nächsten Disziplin ignorieren
                if ignore_section:
                    # Disziplin erkannt → Ausländer-Abschnitt endet
                    if discipline_pattern.match(line):
                        ignore_section = False
                    else:
                        continue
                # Skip team results
                if "Mannschaft" in line or "Mannschaftswertung" in line:
                    continue
                # Skip lines in parentheses (individuals within team)
                if line.startswith("(") and line.endswith(")"):
                    continue
                # Skip Staffel lines
                if re.search(r"\dx\d+", line):
                    continue
                # Discipline detected
                match_d = discipline_pattern.match(line)
                if match_d:
                    current_discipline = match_d.group(0).strip()
                    # Standardisierte Schreibweise (immer Leerzeichen)
                    current_discipline = discipline_standardization.get(current_discipline, current_discipline)

                    continue
                # Result line detected
                match = line_pattern.match(line)
                if match and current_discipline:
                    data = match.groupdict()
                    data.update({
                        "jahr": year,
                        "geschlecht": gender,
                        "altersklasse": age_class,
                        "disziplin": current_discipline
                    })
                    results.append(data)
    return results

# Alle PDFs durchgehen
all_results = []
for pdf_file in pdf_folder.glob("*.pdf"):
    print(f"Processing: {pdf_file.name}")
    all_results.extend(parse_pdf(pdf_file))

# DataFrame
df = pd.DataFrame(all_results)

# Umlaut-/Sonderzeichen konvertieren
df = df.applymap(replace_umlauts)

# Spalten sortieren
df = df[["jahr", "geschlecht", "altersklasse", "disziplin",
         "leistung", "wind", "name", "geburtsjahr",
         "verein", "datum", "ort"]]

# CSV speichern
df.to_csv("list_24.csv", index=False, sep=";")

print("✅ Fertig! Daten gespeichert in: bestenlisten_gesamt.csv")
df.head(10)


<>:14: SyntaxWarning: invalid escape sequence '\d'
<>:14: SyntaxWarning: invalid escape sequence '\d'
/var/folders/zw/nph6kjjs0nxbnglr41zhg5w40000gn/T/ipykernel_22485/1845566143.py:14: SyntaxWarning: invalid escape sequence '\d'
  """line_pattern = re.compile(


Processing: dlv_bestenliste2024_juniorinnen.pdf
Processing: dlv_bestenliste2024_maennliche_jugend_u16.pdf
Processing: dlv_bestenliste2024_weibliche_jugend_u16.pdf
Processing: dlv_bestenliste2024_maenner.pdf
Processing: dlv_bestenliste2024_frauen.pdf
Processing: dlv_bestenliste2024_maennliche_jugend_u20.pdf
Processing: dlv_bestenliste2024_weibliche_jugend_u20.pdf
Processing: dlv_bestenliste2024_weibliche_jugend_u18.pdf
Processing: dlv_bestenliste2024_maennliche_jugend_u18.pdf
Processing: dlv_bestenliste2024_maennliche_jugend_m_14.pdf
Processing: dlv_bestenliste2024_weibliche_jugend_w_14.pdf
Processing: dlv_bestenliste2024_junioren.pdf


/var/folders/zw/nph6kjjs0nxbnglr41zhg5w40000gn/T/ipykernel_22485/1845566143.py:209: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(replace_umlauts)


✅ Fertig! Daten gespeichert in: bestenlisten_gesamt.csv


,jahr,geschlecht,altersklasse,disziplin,leistung,wind,name,geburtsjahr,verein,datum,ort
0,2024,W,U23,100 m,"11,35","+0,2",Sina Kammerschmitt,2003,MTG Mannheim,29.06.2024,Braunschweig
1,2024,W,U23,100 m,"11,37","+1,5",Nele Jaworski,2004,VfL Wolfsburg,06.07.2024,Moenchengladbach
2,2024,W,U23,100 m,"11,37","+1,5",Chelsea Kadiri,2005,Sportclub Magdeburg,06.07.2024,Moenchengladbach
3,2024,W,U23,100 m,"11,43","-0,9",Jolina Ernst,2004,TV Wattenscheid 01,14.07.2024,Rhede
4,2024,W,U23,100 m,"11,49","+1,0",Sherin Kimuanga,2006,SC DHfK Leipzig,22.06.2024,Mannheim
5,2024,W,U23,100 m,"11,53","+0,2",Annika Just,2005,LAC Passau,25.05.2024,Regensburg
6,2024,W,U23,100 m,"11,56","+0,8",Lena Anochili,2007,Hamburger SV,06.07.2024,Moenchengladbach
7,2024,W,U23,100 m,"11,58","+1,5",Viola John,2003,LG Stadtwerke Muenchen,06.07.2024,Moenchengladbach
8,2024,W,U23,100 m,"11,62","+0,2",Charlotte Riedel,2005,SC DHfK Leipzig,25.05.2024,Regensburg
9,2024,W,U23,100 m,"11,66","+0,8",Line Schroeder,2004,Hamburger SV,20.07.2024,Breda (NED)
